In [33]:
import requests
import urllib
import ftplib

from datetime import datetime, date

In [2]:
# Reference docs for the format and organization of ADP VIIRS data 
# https://www.star.nesdis.noaa.gov/smcd/spb/aq/AerosolWatch/docs/EPS_ADP_Users_Guide_V1_Sep2020.pdf

# Bounding box values for CA
# https://anthonylouisdagostino.com/bounding-boxes-for-all-us-states/
# original bounding box (lat, lon)
UPPER_RIGHT = (43, -114)
LOWER_LEFT = (32, -125)

# Need to extend the bounding box region, since the swath width of VIIRS granule is 3060 km.
LAT_OFFSET = 10
LON_OFFSET = 20

MAXLAT = UPPER_RIGHT[0] + LAT_OFFSET
MAXLON = UPPER_RIGHT[1] + LON_OFFSET
MINLAT = LOWER_LEFT[0] - LAT_OFFSET
MINLON = LOWER_LEFT[1] - LON_OFFSET

# API URL for OrbNav API
ORBNAV_API_URL = "https://sips.ssec.wisc.edu/orbnav/api/v1/boxtimes.json?"

# Satellite IDs used in the OrbNav API
SATELLITE_ID_MAP = {
    "NOAA_20": 43013,
    "SNPP": 37849
}

# FTP hostname 
FTP_HOSTNAME = "ftp.star.nesdis.noaa.gov"

# Filepath on the FTP server for the two VIIRS satellites
SATELLITE_FTP_PATH_MAP = {
    "NOAA_20": "/pub/smcd/hzhang/VIIRS_EPS_NRT/ADP_N20/CONUS/",
    "SNPP": "/pub/smcd/hzhang/VIIRS_EPS_NRT/ADP/CONUS/"
}

# Paste in browser to view files (sample) 
FTP_TEST = 'ftp.star.nesdis.noaa.gov/pub/smcd/hzhang/VIIRS_EPS_NRT/ADP_N20/CONUS/'

In [37]:
def _fetch_overpass_times(satellite: str, date: datetime = datetime.utcnow()):
    # Returns a list of (start, end) datetime objects for a satellite using the OrbNav API
    satellite_id = SATELLITE_ID_MAP[satellite]
    ur = str(MAXLAT) + ", " + str(MAXLON)
    ll = str(MINLAT) + ", " + str(MINLON)
    args = {
        "sat": satellite_id,
        "start": get_utc_start(date),
        "end": get_utc_end(date),
        "ll": ll,
        "ur": ur
    }
    request_url = ORBNAV_API_URL + urllib.parse.urlencode(args)
    resp = requests.get(request_url)
    if resp.status_code != 200:
        return
    return _process_resp(resp.json())

def _get_utc_start(curr_date: datetime):
    # return midnight timestamp of  date
    midnight = curr_date.replace(hour=0, minute=0, second=0, microsecond=0)
    return midnight.strftime("%Y-%m-%dT%H:%M:%SZ")

def _get_utc_end(curr_date: datetime):
    # return latest timestamp of  date
    midnight = curr_date.replace(hour=23, minute=59, second=59, microsecond=0)
    return midnight.strftime("%Y-%m-%dT%H:%M:%SZ")

def _process_resp(overpass_times_json):
    # processes the request and creates a list of (start, end) datetime objects that correspond to when the satellite
    # was in the bounding box
    overpass_times = []
    for data in overpass_times_json["data"]:
        start_timestamp = data[0][0]
        end_timestamp = data[1][0]
        start = datetime.strptime(start_timestamp, '%Y-%m-%dT%H:%M:%SZ')
        end = datetime.strptime(end_timestamp, '%Y-%m-%dT%H:%M:%SZ')
        overpass_times.append([start, end])
    return overpass_times

def _is_valid_granule(filename, overpass_times):
    observation_timestamp = get_observation_timestamp(filename)
    for overpass in overpass_times:
        start, end = overpass
        if start <= observation_timestamp <= end: 
            return True
    return False

def _get_observation_timestamp(filename):
    # Returns a datetime object corresponding to the file observation time
    observation_timestamp = filename.split("_")[3][1:]
    observation_datetime = datetime.strptime(observation_timestamp, '%Y%m%d%H%M%S%f').replace(microsecond=0)
    return observation_datetime

# MAIN DRIVER FUNCTION
def download_granules(satellite: str, download_dir = "", date: datetime = datetime.utcnow()):
    # Downloads the granules from the FTP server for a given satellite for timestamps covering California in download_dir
    ftp_path = SATELLITE_FTP_PATH_MAP[satellite]
    ftp_path += date.strftime("%Y%m%d")
    
    overpass_times = _fetch_overpass_times(satellite, date)
    
    with ftplib.FTP(FTP_HOSTNAME) as ftp:
        try:
            ftp.login()
            ftp.cwd(ftp_path)
            files = []
            ftp.dir(files.append)
            for file in files:
                filename = file.split(" ")[-1]
                if (_is_valid_granule(filename, overpass_times)):
                    with open("/".join([download_dir, filename]), 'wb') as f:
                        print("Downloading %s..." % filename)
                        ftp.retrbinary('RETR ' + filename, f.write)
        except ftplib.all_errors as e:
            print('FTP error:', e)

In [38]:
dec1 = datetime(2020, 12, 1)

In [39]:
fetch_overpass_times("NOAA_20", dec1)

[[datetime.datetime(2020, 12, 1, 8, 35, 20),
  datetime.datetime(2020, 12, 1, 8, 44, 13)],
 [datetime.datetime(2020, 12, 1, 10, 16, 50),
  datetime.datetime(2020, 12, 1, 10, 25, 43)],
 [datetime.datetime(2020, 12, 1, 19, 56, 30),
  datetime.datetime(2020, 12, 1, 20, 5, 24)],
 [datetime.datetime(2020, 12, 1, 21, 38),
  datetime.datetime(2020, 12, 1, 21, 46, 54)]]

In [40]:
download_granules("NOAA_20", "data",  dec1)

In [31]:
fetch_overpass_times("SNPP", dec1)

[[datetime.datetime(2020, 12, 1, 9, 25, 54),
  datetime.datetime(2020, 12, 1, 9, 34, 47)],
 [datetime.datetime(2020, 12, 1, 11, 7, 24),
  datetime.datetime(2020, 12, 1, 11, 16, 17)],
 [datetime.datetime(2020, 12, 1, 19, 10, 52),
  datetime.datetime(2020, 12, 1, 19, 14, 28)],
 [datetime.datetime(2020, 12, 1, 20, 47, 4),
  datetime.datetime(2020, 12, 1, 20, 55, 58)],
 [datetime.datetime(2020, 12, 1, 22, 28, 34),
  datetime.datetime(2020, 12, 1, 22, 34, 4)]]

In [32]:
download_granules("SNPP", "data",  dec1)